# 02. LSTM (Long Short-Term Memory): 절대 까먹지 않는 기억력
### 과목: NLP Sequence Modeling
---

## 1. RNN의 한계와 LSTM의 등장
지난 시간에 배운 **RNN**은 "건망증"이 심했습니다.
- **상황**: 문장이 100단어가 넘어가면, 맨 첫 단어를 까먹어버립니다.
- **해결책**: "중요한 건 오래 기억하고, 쓸데없는 건 빨리 잊어버리자!"

## 2. LSTM 핵심 원리: "3명의 수문장 (Gates)"
LSTM에는 기억을 관리하는 3개의 문(Gate)이 있습니다.

1.  **망각 게이트 (Forget Gate)**: "이거 쓸모없는 정보네? 버려!" 
    - 예: '그는 어제 밥을 먹었다.' -> '어제'라는 정보는 이제 필요 없으니 삭제.
2.  **입력 게이트 (Input Gate)**: "오, 이거 중요한 정보네? 기억해!"
    - 예: '내일은 시험이다.' -> '시험'이라는 정보는 중요하니 저장.
3.  **출력 게이트 (Output Gate)**: "지금은 이 정보를 내보내자!"
    - 예: 누가 "시험 언제야?"라고 물으면 '내일'이라는 정보를 꺼냄.

> **결론**: LSTM은 **Cell State(컨베이어 벨트)**를 통해 기억을 아주 오랫동안 운반합니다.

---

## 3. 실습: PyTorch로 LSTM 구조 확인하기
코드는 RNN과 거의 똑같습니다. 단지 `nn.RNN` 대신 `nn.LSTM`을 쓸 뿐입니다.

In [ ]:
import torch
import torch.nn as nn

# --- [1] 데이터 준비 ---
batch_size = 2
seq_len = 3
input_size = 4
hidden_size = 5

x = torch.randn(batch_size, seq_len, input_size)
print(f"입력 데이터: {x.shape}")

# --- [2] LSTM 모델 생성 ---
# batch_first=True: (배치, 시간, 특징)
lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

# --- [3] 실행 (Forward) ---
# RNN과 다른 점: 리턴값이 하나 더 있다! (Cell State)
# output: 매 시점의 출력
# hidden: 단기 기억 (Short-term Memory)
# cell: 장기 기억 (Long-term Memory) -> 이게 핵심!
output, (hidden, cell) = lstm(x)

print("\n--- 결과 확인 ---")
print(f"Output: {output.shape} (모든 시점의 출력)")
print(f"Hidden State: {hidden.shape} (최종 단기 기억)")
print(f"Cell State: {cell.shape} (최종 장기 기억) ★중요")

---
## 4. 실전: IMDB 감성 분석 (업그레이드 버전)
RNN때와 같은 데이터를 쓰지만, 이번엔 LSTM으로 성능을 높여봅니다.

In [ ]:
from tensorflow.keras.datasets import imdb
import torch.nn.functional as F

# 1. 데이터 로드
vocab_size = 300
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# 2. 데이터 줄이기 (실습용)
X_train, y_train = X_train[:15000], y_train[:15000]
X_test, y_test = X_test[:10000], y_test[:10000]

# 3. 패딩 (길이 맞추기)
seq_len = 100
def pad_sequences(sequences, max_len):
    padded_list = []
    for seq in sequences:
        seq_tensor = torch.tensor(seq, dtype=torch.long)
        if len(seq_tensor) < max_len:
            padded = F.pad(seq_tensor, (0, max_len - len(seq_tensor)), value=0)
        else:
            padded = seq_tensor[:max_len]
        padded_list.append(padded)
    return torch.stack(padded_list)

X_train_padded = pad_sequences(X_train, seq_len)
X_test_padded = pad_sequences(X_test, seq_len)

# 4. 원-핫 인코딩
X_train_onehot = F.one_hot(X_train_padded, num_classes=vocab_size).float()
X_test_onehot = F.one_hot(X_test_padded, num_classes=vocab_size).float()

In [ ]:
import torch.nn as nn

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        # LSTM 층 선언
        self.lstm = nn.LSTM(
            input_size=input_dim, 
            hidden_size=hidden_dim, 
            batch_first=True
        )
        # 출력 층
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # LSTM 실행
        # output: 전체 문장의 흐름
        # hidden: 단기 기억 (마지막 상태)
        # cell: 장기 기억 (마지막 상태)
        output, (hidden, cell) = self.lstm(x)
        
        # 우리는 '마지막 단기 기억(hidden)'을 써서 판별할 겁니다.
        # hidden은 (층수, 배치, 차원)이므로 마지막 층([-1])을 가져옵니다.
        last_hidden = hidden[-1]
        
        # 긍정/부정 판단
        score = self.fc(last_hidden)
        return score

# 모델 생성 (RNN보다 hidden_size를 좀 더 키워봄)
model = LSTMNet(vocab_size, hidden_dim=16, output_dim=1)
print("LSTM 모델 준비 완료!")

In [ ]:
# 학습 코드는 RNN과 동일합니다.
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

y_train_tensor = torch.tensor(y_train).float().unsqueeze(1)
train_dataset = TensorDataset(X_train_onehot, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("LSTM 학습 시작...")
model.train()
for epoch in range(3): # 3 에폭만 돌려봅시다
    total_loss = 0
    for inputs, targets in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

### 🌟 오늘의 결론
1. **RNN**은 잘 까먹는데, **LSTM**은 장기 기억(Cell State)이 있어서 긴 글도 잘 이해한다.
2. PyTorch에서는 그냥 `nn.RNN`을 `nn.LSTM`으로만 바꾸면 된다.
3. 성능은 훨씬 좋다!